In [6]:
import torch.nn as nn
import torchvision
import torch
import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib
import matplotlib.pyplot as plt
import pickle
%run nets.ipynb
%run plot_graphs.ipynb
import shutil
from os import makedirs

In [11]:
def loadData(path):
    data = torch.load(path + '/data_train_train.pt').float()
    labels_ = torch.load(path + '/labels_train_train.pt').long()
    dataTest = torch.load(path + '/data_train_eval.pt').float()
    labelsTest = torch.load(path + '/labels_train_eval.pt').long()
    poseTrain = torch.load(path + '/data_train_train_key.pt').float()
    poseTest = torch.load(path + '/data_train_eval_key.pt').float()
    poseTrain1 = torch.load(path + '/data_train_train_key_1.pt').float()
    poseTest1 = torch.load(path + '/data_train_eval_key_1.pt').float()
    return data, labels_, dataTest, labelsTest, poseTrain, poseTest, poseTrain1, poseTest1



def train(modelName, learning_rate, weightedHier = None, num_epochs = 3, batch_size = 2, dropout = 0.0,
         reproducibility = True, buildGraphs = False, ckp_path = None):

    
    #init
    name = modelName + "_" + str(learning_rate) + "_" + str(dropout) + str(num_epochs) 
    checkpoint_path = "./checkpoints/" + name
    best_model_path = "./bestModels/" + name
    try:
        makedirs('./checkpoints')
    except Exception as e:
        None
    try:
        makedirs('./bestModels')
    except Exception as e:
        None
    bestAcc = 0.0
    
    
    #reproducibility
    if reproducibility == True:
        torch.manual_seed(30)
        torch.cuda.manual_seed(30)
        np.random.seed(30)
        torch.backends.cudnn.deterministic = True
    
    device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
    print('Using device:', device)

    # initialize network
    print(modelName)
    if modelName == "cnn":
        model = myModel("cnn",dropout = 0.2)
    elif modelName == "hier":
        model = myModel("hier",dropout = 0.2)
    elif modelName == "combined1":
        model = myModel("combined1",dropout = 0.2)
    elif modelName == "combined2":
        model = myModel("combined2",dropout = 0.2)
    elif modelName == "pose":
        model = Net_pose()
    elif modelName == "dense":
        modelName = "cnn"
        model = DenseNet3(82, "cnn", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    elif modelName == "dense_hier":
        modelName = "hier"
        model = DenseNet3(82, "hier", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    elif modelName == "dense_combined1":
        modelName = "combined1"
        model = DenseNet3(82, "combined1", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    elif modelName == "dense_combined2":
        modelName = "combined2"
        model = DenseNet3(82, "combined2", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    else:
        print("model name does not exist")
        return 0
    model.to(device)
    
    
    #set hyperparameter
    if weightedHier == None:
        weightedHier = [1/3,1/3,1/3]
    error = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, threshold=0.001, factor=0.5)

    # prepare train
    accList1 = []
    accList2 = []
    accList3 = []
    start_epoch = 0
    #mesures true positive,false positive and false negative (checkpoint not working for metrics right now)
    metrics = np.zeros((num_epochs,82,3)) 
    
    num_Batches = int(data.shape[0] / batch_size)
    if data.shape[0] % batch_size != 0:
        num_Batches += 1

    #load model from checkpoint and continue training
    if ckp_path != None:
        model, optimizer, scheduler, start_epoch, bestAcc, accList1, accList2, accList3 = load_ckp(ckp_path, model, optimizer, scheduler)
        num_epochs -= start_epoch
        
        
    #train
    for epoch in range(num_epochs):
        model.train()
        for i in range(num_Batches):
            #get the correct slice of the data
            cutBatch = (i+1)*batch_size
            if i == num_Batches - 1:
                cutBatch = None
            
            #get the correct data for the model type
            labels = labels_[i*batch_size:cutBatch]
            if modelName != "pose":
                images = data[i*batch_size:cutBatch] 
            if modelName == "combined1" or modelName == "pose": 
                keys = poseTrain[i*batch_size:cutBatch] 
            elif modelName == "combined2":
                keys = poseTrain1[i*batch_size:cutBatch]
            
            if modelName == "combined1" or modelName == "pose":
                keys = Variable(keys.view(keys.shape[0],51))
            if modelName != "pose":
                train = Variable(images.view(images.shape[0],100,100,3))
                train = torch.transpose(train, 1, 3)
                train = torch.transpose(train, 2, 3)
            if modelName == "combined2":
                keys = Variable(keys.view(keys.shape[0],1,100,100))
                train = torch.cat((train, keys), dim=1)
            labels1 = Variable(labels[:,0])
            labels2 = Variable(labels[:,1])
            labels3 = Variable(labels[:,2])
            
            if modelName != "pose":
                train = train.to(device).to(torch.float)
            if modelName != "cnn" and modelName != "hier":
                keys = keys.to(device).to(torch.float)
            labels1, labels2, labels3 = (
                labels1.to(device),
                labels2.to(device),
                labels3.to(device),
            )
            
            #run data trough model
            optimizer.zero_grad()
            if modelName == "cnn":
                outputs = model(train)
                loss = error(outputs, labels3)
            elif modelName == "pose":
                outputs = model(keys)
                loss = error(outputs, labels3)
            else:
                if modelName == "combined1":
                    a,b,outputs = model(train,keys)
                else:
                    a,b,outputs = model(train)
                loss1 = error(a, labels1)
                loss2 = error(b, labels2)
                loss3 = error(outputs, labels3)
                loss = weightedHier[0]*loss1 + weightedHier[1]*loss2 + weightedHier[2]*loss3
            loss.backward()
            optimizer.step()
            
        # eval
        with torch.no_grad():
            #prepare eval
            model.eval()
            correct1 = 0
            total = 0
            correct2 = 0
            correct3 = 0
            totalLoss = 0
            
            #get the correct slice of the data type
            num_Batches_test = int(dataTest.shape[0] / batch_size)
            if dataTest.shape[0] % batch_size != 0:
                num_Batches_test += 1
            for i in range(num_Batches_test):
                cutBatch = (i+1)*batch_size
                if i == num_Batches - 1:
                    cutBatch = None
                    
                #get the correct data for model
                labels = labelsTest[i*batch_size:cutBatch]
                if modelName != "pose":
                    images = dataTest[i*batch_size:cutBatch]
                if modelName == "combined1" or modelName == "pose": 
                    keysLabel = poseTest[i*batch_size:cutBatch] 
                elif modelName == "combined2":
                    keysLabel = poseTest1[i*batch_size:cutBatch]

                if modelName == "combined1" or modelName == "pose":
                    keysLabel = Variable(keysLabel.view(keysLabel.shape[0],51))
                if modelName != "pose":
                    test = Variable(images.view(images.shape[0],100,100,3))
                    test = torch.transpose(test, 1, 3)
                    test = torch.transpose(test, 2, 3)
                if modelName == "combined2":
                    keysLabel = Variable(keysLabel.view(keysLabel.shape[0],1,100,100))
                    test = torch.cat((test, keysLabel), dim=1)
                labels1 = Variable(labels[:,0])
                labels2 = Variable(labels[:,1])
                labels3 = Variable(labels[:,2])

                if modelName != "pose":
                    test = test.to(device).to(torch.float)
                if modelName != "cnn" and modelName != "hier":
                    keysLabel = keysLabel.to(device).to(torch.float)
                labels1, labels2, labels3 = (
                    labels1.to(device),
                    labels2.to(device),
                    labels3.to(device),
                )

                #compute accuracies, loss and metrices
                if modelName == "cnn":
                    outputs = model(test)
                    loss = error(outputs, labels3)
                elif modelName == "pose":
                    outputs = model(keysLabel)
                    loss = error(outputs, labels3)
                else:
                    if modelName == "combined1":
                        a,b,outputs = model(test,keysLabel)
                    else:
                        a,b,outputs = model(test)
                    predicted1 = torch.max(a.data, 1)[1]
                    predicted2 = torch.max(b.data, 1)[1]
                    correct1 += (predicted1 == labels1).sum()
                    correct2 += (predicted2 == labels2).sum()
                    loss1 = error(a, labels1)
                    loss2 = error(b, labels2)
                    loss3 = error(outputs, labels3)
                    loss = weightedHier[0]*loss1 + weightedHier[1]*loss2 + weightedHier[2]*loss3
                predicted3 = torch.max(outputs.data, 1)[1]  
                total += len(labels3)
                correct3 += (predicted3 == labels3).sum()
                totalLoss += loss.sum()

                for pred,lab in zip(predicted3,labels3):
                    if pred == lab:
                        metrics[epoch][pred][0] += 1
                    else:
                        metrics[epoch][pred][1] += 1
                        metrics[epoch][lab][2] += 1
            if modelName != "cnn" and modelName != "pose":
                accuracy1 = 100 * correct1 / float(total)
                accuracy2 = 100 * correct2 / float(total)
            accuracy3 = 100 * correct3 / float(total)
            epochLoss = totalLoss / float(total)
            
            scheduler.step(epochLoss)
            
            #print progress
            if modelName != "cnn" and modelName != "pose":
                print('Epoch: {}  accuracy1: {}  accuracy2: {}  accuracy3: {}  overall loss: {}'.format(epoch+1+start_epoch, 
                                                                            accuracy1,accuracy2,accuracy3,epochLoss))
                accList1.append(accuracy1.item())
                accList2.append(accuracy2.item())
            else:
                print('Epoch: {}  accuracy: {}  loss: {}'.format(epoch+1+start_epoch, accuracy3,epochLoss))
            accList3.append(accuracy3.item())


        # save best model
        if accuracy3 >= bestAcc:
            # save as best model
            torch.save(model, "./bestModels/" + name)
            bestAcc = accuracy3

            
        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'bestAcc': bestAcc,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'accList1': accList1,
            'accList2': accList2,
            'accList3': accList3,
        }
        # save checkpoint
        torch.save(checkpoint, checkpoint_path)
            
            
            
    #plot the accuracies
    if buildGraphs == True: 
        try:
            makedirs('./plots')
        except Exception as e:
            None
        if ckp_path == None:
            metrics = metrics.reshape(num_epochs,82*3)
            np.savetxt("./plots/" + name + "_metrics_.txt", metrics, fmt='%d')
        if modelName != "CNN" and modelName != "pose":
            plot_history([accList1, accList2, accList3], name)
        else:
            plot_history([accList3], name)
        with open("./plots/" + name + "_acc.txt", "wb") as fp:   #Pickling
            pickle.dump([accList1, accList2, accList3], fp)
        
    #save final model
    try:
        makedirs('./models')
    except Exception as e:
        None
    torch.save(model, "./models/" + name)
    return 0




def load_ckp(checkpoint_path, model, optimizer, scheduler):
    """
    checkpoint_path: path to save checkpoint
    model: model to load checkpoint parameters into       
    optimizer: optimizer to use
    scheduler: scheduler to use
    """
    # load check point
    checkpoint = torch.load(checkpoint_path)
    # initialize
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    bestAcc = checkpoint['bestAcc']
    return model, optimizer, scheduler, checkpoint['epoch'], bestAcc, checkpoint['accList1'],checkpoint['accList2'], checkpoint['accList3']

In [3]:
#load entire data
data, labels_, dataTest, labelsTest, poseTrain, poseTest, poseTrain1, poseTest1 = loadData("./allData")

In [10]:
"""
Runs
"""
#train models
train("dense_combined1", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)
train("dense_combined2", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)
train("dense", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)

Using device: cuda:0
dense_combined1
Epoch: 1  accuracy1: 32.62883758544922  accuracy2: 12.275622367858887  accuracy3: 12.217719078063965  overall loss: 0.18553267419338226
Epoch: 2  accuracy1: 37.17428970336914  accuracy2: 20.208454132080078  accuracy3: 13.144181251525879  overall loss: 0.1653822660446167
Epoch: 3  accuracy1: 42.55935287475586  accuracy2: 25.07238006591797  accuracy3: 16.41575050354004  overall loss: 0.1561366766691208
Epoch: 4  accuracy1: 43.94904708862305  accuracy2: 29.183555603027344  accuracy3: 18.09496307373047  overall loss: 0.15156812965869904
Epoch: 5  accuracy1: 43.42790985107422  accuracy2: 28.517662048339844  accuracy3: 19.80312728881836  overall loss: 0.15192696452140808
Epoch: 6  accuracy1: 44.296470642089844  accuracy2: 29.878402709960938  accuracy3: 21.453388214111328  overall loss: 0.1494004875421524
Epoch: 7  accuracy1: 46.467864990234375  accuracy2: 32.513031005859375  accuracy3: 22.5825138092041  overall loss: 0.14601850509643555
Epoch: 8  accuracy

Epoch: 17  accuracy1: 50.289520263671875  accuracy2: 39.577301025390625  accuracy3: 4.89287805557251  overall loss: 0.17207182943820953
Epoch    18: reducing learning rate of group 0 to 3.7500e-04.
Epoch: 18  accuracy1: 50.028953552246094  accuracy2: 39.490447998046875  accuracy3: 4.95078182220459  overall loss: 0.1723647564649582
Epoch: 19  accuracy1: 49.79733657836914  accuracy2: 39.953678131103516  accuracy3: 4.95078182220459  overall loss: 0.1717727780342102
Epoch: 20  accuracy1: 50.43428039550781  accuracy2: 39.490447998046875  accuracy3: 4.89287805557251  overall loss: 0.17219285666942596
Epoch: 21  accuracy1: 50.463233947753906  accuracy2: 39.751014709472656  accuracy3: 5.06658935546875  overall loss: 0.17268545925617218
Epoch    22: reducing learning rate of group 0 to 1.8750e-04.
Epoch: 22  accuracy1: 50.289520263671875  accuracy2: 39.8668212890625  accuracy3: 5.037637710571289  overall loss: 0.1730097234249115
Epoch: 23  accuracy1: 50.43428039550781  accuracy2: 39.98263168334

KeyboardInterrupt: 

In [4]:
train("dense_hier", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)

Using device: cuda:0
dense_hier
Epoch: 1  accuracy1: 37.463809967041016  accuracy2: 22.061378479003906  accuracy3: 3.9374639987945557  overall loss: 0.18021434545516968
Epoch: 2  accuracy1: 39.085121154785156  accuracy2: 24.522293090820312  accuracy3: 4.574406623840332  overall loss: 0.17845164239406586
Epoch: 3  accuracy1: 43.65952682495117  accuracy2: 28.25709342956543  accuracy3: 4.3427910804748535  overall loss: 0.17307797074317932
Epoch: 4  accuracy1: 44.151710510253906  accuracy2: 29.646787643432617  accuracy3: 5.124493598937988  overall loss: 0.17268535494804382
Epoch: 5  accuracy1: 40.2142448425293  accuracy2: 22.72727394104004  accuracy3: 3.705848455429077  overall loss: 0.1876976490020752
Epoch: 6  accuracy1: 39.953678131103516  accuracy2: 25.477706909179688  accuracy3: 3.590040683746338  overall loss: 0.18564419448375702
Epoch: 7  accuracy1: 47.539085388183594  accuracy2: 34.018531799316406  accuracy3: 4.863925933837891  overall loss: 0.16798044741153717
Epoch: 8  accuracy1:

0

In [5]:
train("dense_combined1", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 100, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True, ckp_path = "./checkpoints/dense_combined1_0.003_0.0")

Using device: cuda:0
dense_combined1
Epoch: 3  accuracy1: 34.94499206542969  accuracy2: 20.121599197387695  accuracy3: 11.957151412963867  overall loss: 0.1697954684495926
Epoch: 4  accuracy1: 42.067169189453125  accuracy2: 23.885351181030273  accuracy3: 15.489288330078125  overall loss: 0.15953503549098969
Epoch: 5  accuracy1: 42.7041130065918  accuracy2: 26.34626579284668  accuracy3: 18.008106231689453  overall loss: 0.1567986011505127
Epoch     6: reducing learning rate of group 0 to 1.5000e-03.
Epoch: 6  accuracy1: 43.6884765625  accuracy2: 27.533294677734375  accuracy3: 19.658367156982422  overall loss: 0.15303201973438263
Epoch: 7  accuracy1: 42.79096984863281  accuracy2: 27.967575073242188  accuracy3: 22.292993545532227  overall loss: 0.15093636512756348
Epoch: 8  accuracy1: 45.888824462890625  accuracy2: 33.584251403808594  accuracy3: 23.972206115722656  overall loss: 0.14508391916751862
Epoch: 9  accuracy1: 45.36769104003906  accuracy2: 32.5419807434082  accuracy3: 24.69600486

Epoch    58: reducing learning rate of group 0 to 1.8311e-07.
Epoch: 58  accuracy1: 49.07353973388672  accuracy2: 37.89809036254883  accuracy3: 30.312681198120117  overall loss: 0.14087139070034027
Epoch: 59  accuracy1: 49.044586181640625  accuracy2: 37.840187072753906  accuracy3: 30.312681198120117  overall loss: 0.14089810848236084
Epoch: 60  accuracy1: 49.07353973388672  accuracy2: 37.92704391479492  accuracy3: 30.283729553222656  overall loss: 0.14088718593120575
Epoch: 61  accuracy1: 49.044586181640625  accuracy2: 37.869136810302734  accuracy3: 30.341632843017578  overall loss: 0.14086836576461792
Epoch    62: reducing learning rate of group 0 to 9.1553e-08.
Epoch: 62  accuracy1: 49.0156364440918  accuracy2: 37.92704391479492  accuracy3: 30.254777908325195  overall loss: 0.14086119830608368
Epoch: 63  accuracy1: 49.13144302368164  accuracy2: 37.782283782958984  accuracy3: 30.283729553222656  overall loss: 0.14094345271587372
Epoch: 64  accuracy1: 49.044586181640625  accuracy2: 37.

0

In [4]:
train("cnn", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)
train("hier", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)
train("combined1", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)
train("combined2", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 40, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)

Using device: cuda:0
cnn
Epoch: 1  accuracy: 14.44701862335205  loss: 0.22817900776863098
Epoch: 2  accuracy: 22.177186965942383  loss: 0.19990171492099762
Epoch: 3  accuracy: 27.909669876098633  loss: 0.18494626879692078
Epoch: 4  accuracy: 28.778228759765625  loss: 0.18209215998649597
Epoch: 5  accuracy: 32.194557189941406  loss: 0.17134562134742737
Epoch: 6  accuracy: 34.85813522338867  loss: 0.16671264171600342
Epoch: 7  accuracy: 35.552982330322266  loss: 0.16321682929992676
Epoch: 8  accuracy: 35.06079864501953  loss: 0.1666542887687683
Epoch: 9  accuracy: 35.78459930419922  loss: 0.16289980709552765
Epoch: 10  accuracy: 37.02953338623047  loss: 0.1642378717660904
Epoch: 11  accuracy: 38.30341720581055  loss: 0.16358496248722076
Epoch: 12  accuracy: 38.01389694213867  loss: 0.17050673067569733
Epoch    13: reducing learning rate of group 0 to 1.5000e-03.
Epoch: 13  accuracy: 37.69542694091797  loss: 0.17087312042713165
Epoch: 14  accuracy: 41.71974563598633  loss: 0.1645655930042

Epoch: 36  accuracy1: 57.75912094116211  accuracy2: 56.07991027832031  accuracy3: 46.81528854370117  overall loss: 0.12544327974319458
Epoch    37: reducing learning rate of group 0 to 2.3438e-05.
Epoch: 37  accuracy1: 57.90388107299805  accuracy2: 56.195716857910156  accuracy3: 47.133758544921875  overall loss: 0.12557461857795715
Epoch: 38  accuracy1: 58.048641204833984  accuracy2: 56.10886001586914  accuracy3: 46.81528854370117  overall loss: 0.12534838914871216
Epoch: 39  accuracy1: 58.048641204833984  accuracy2: 56.340476989746094  accuracy3: 47.04690170288086  overall loss: 0.12549743056297302
Epoch: 40  accuracy1: 57.75912094116211  accuracy2: 56.25362014770508  accuracy3: 47.10480880737305  overall loss: 0.1256469041109085
Using device: cuda:0
combined1
Epoch: 1  accuracy1: 39.5483512878418  accuracy2: 21.598148345947266  accuracy3: 6.079907417297363  overall loss: 0.17182067036628723
Epoch: 2  accuracy1: 45.39664077758789  accuracy2: 30.225826263427734  accuracy3: 13.665315628

Epoch: 13  accuracy1: 52.4898681640625  accuracy2: 49.160396575927734  accuracy3: 40.04053497314453  overall loss: 0.12000084668397903
Epoch    14: reducing learning rate of group 0 to 1.5000e-03.
Epoch: 14  accuracy1: 52.605674743652344  accuracy2: 49.420963287353516  accuracy3: 41.024898529052734  overall loss: 0.1208060085773468
Epoch: 15  accuracy1: 55.2982063293457  accuracy2: 53.01100158691406  accuracy3: 44.06485366821289  overall loss: 0.116414375603199
Epoch: 16  accuracy1: 54.80602264404297  accuracy2: 51.56340408325195  accuracy3: 43.22524642944336  overall loss: 0.11825385689735413
Epoch: 17  accuracy1: 54.86392593383789  accuracy2: 52.8372917175293  accuracy3: 43.60162353515625  overall loss: 0.12142407149076462
Epoch    18: reducing learning rate of group 0 to 7.5000e-04.
Epoch: 18  accuracy1: 55.9640998840332  accuracy2: 52.663578033447266  accuracy3: 43.74638366699219  overall loss: 0.12275813519954681
Epoch: 19  accuracy1: 54.05327224731445  accuracy2: 54.2269859313964

0

In [11]:
train("pose", 0.003, weightedHier = [1/3,1/3,1/3], num_epochs = 100, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True, ckp_path = "./checkpoints/pose_0.003_0.0400")
train("pose", 0.0001, weightedHier = [1/3,1/3,1/3], num_epochs = 100, batch_size = 16, dropout = 0.0,
         reproducibility = True, buildGraphs = True)

Using device: cuda:0
pose
Epoch: 83  accuracy: 29.47307586669922  loss: 0.23532822728157043
Epoch: 84  accuracy: 29.47307586669922  loss: 0.23532825708389282
Epoch: 85  accuracy: 29.47307586669922  loss: 0.2353283166885376
Epoch: 86  accuracy: 29.47307586669922  loss: 0.2353283315896988
Epoch: 87  accuracy: 29.47307586669922  loss: 0.2353283166885376
Epoch: 88  accuracy: 29.47307586669922  loss: 0.23532846570014954
Epoch: 89  accuracy: 29.47307586669922  loss: 0.2353285402059555
Epoch: 90  accuracy: 29.47307586669922  loss: 0.23532865941524506
Epoch: 91  accuracy: 29.47307586669922  loss: 0.23532871901988983
Epoch: 92  accuracy: 29.47307586669922  loss: 0.2353287786245346
Epoch: 93  accuracy: 29.47307586669922  loss: 0.23532874882221222
Epoch: 94  accuracy: 29.47307586669922  loss: 0.23532871901988983
Epoch: 95  accuracy: 29.47307586669922  loss: 0.23532883822917938
Epoch: 96  accuracy: 29.47307586669922  loss: 0.23532900214195251
Epoch: 97  accuracy: 29.47307586669922  loss: 0.2353289

Epoch: 95  accuracy: 32.94730758666992  loss: 0.17707940936088562
Epoch: 96  accuracy: 32.94730758666992  loss: 0.17707926034927368
Epoch: 97  accuracy: 32.94730758666992  loss: 0.17707930505275726
Epoch: 98  accuracy: 32.94730758666992  loss: 0.1770792454481125
Epoch: 99  accuracy: 32.976261138916016  loss: 0.1770792007446289
Epoch: 100  accuracy: 32.976261138916016  loss: 0.1770792156457901


0